In [ ]:
#!pip3 install tqdm
from post_processing import *
from const import ROOT
from const import *
from PIL import ImageOps
def tint_image(src, color="#FFFFFF"):
    src.load()
    r, g, b, alpha = src.split()
    gray = ImageOps.grayscale(src)
    result = ImageOps.colorize(gray, (0, 0, 0, 0), color) 
    result.putalpha(alpha)
    return result
import pandas as pd

In [ ]:
df = pd.DataFrame(columns=['n','list'])

In [ ]:
#ROOT='/Users/alexanderliao/data/'
ROOT='/home/alexanderliao/data/Kaggle/competitions/tgs-salt-identification-challenge/'
#ROOT='/data/kaggle/salt/'
train_path = os.path.join(ROOT, 'train')
train_file_list = glob.glob(os.path.join(train_path, 'images', '*.png'))
train_file_list = [f.split('/')[-1].split('.')[0] for f in train_file_list]
test_path = os.path.join(ROOT, 'test')
test_file_list = glob.glob(os.path.join(test_path, 'images', '*.png'))
test_file_list = [f.split('/')[-1].split('.')[0] for f in test_file_list]

imgs=[]
ids=[]
test=[]
for mask in tqdm(train_file_list):
    img = Image.open(ROOT+"train/images/"+mask+".png").convert('LA') 
    arr= np.array(img)[:,:,0]
    imgs.append(arr)
    ids.append(mask)
    test.append(0)
for mask in tqdm(test_file_list):
    img = Image.open(ROOT+"test/images/"+mask+".png").convert('LA') 
    arr= np.array(img)[:,:,0]
    imgs.append(arr)
    ids.append(mask)
    test.append(1)

pickle.dump( imgs, open( "all_image.p", "wb" ) )
pickle.dump( ids, open( "all_ids.p", "wb" ) )
pickle.dump( test, open( "train_test.p", "wb" ) )

img=Image.open('raw/'+str(1)+".png").convert('LA')
arr= np.array(img)[:,:,0]
arr.shape
img_arr=[]

for i in range(int(arr.shape[0]/101)):
    row=[]
    for j in range(int(arr.shape[1]/101)):
        row.append(arr[i*101:(i+1)*101,j*101:(j+1)*101])
    img_arr.append(row)
        

i = 3
j = 4
diffs=[]
for k in range(len(imgs)):
    diffs.append(np.square(imgs[k]-img_arr[i][j]).mean())
    """
    if np.array_equal(imgs[k] , np.fliplr(img_arr[i][j])):
        print((i,j))
        break
    if np.array_equal(imgs[k] , np.flipud(img_arr[i][j])):
        print((i,j))
        break
    if np.array_equal(imgs[k] , np.fliplr(np.flipud(img_arr[i][j]))):
        print((i,j))
        break
    """

In [ ]:
def is_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(101):
        if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[0,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0

def down_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(50,101):
        if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[50,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0

def find_cols(img):
    row = img[100,:]
    #print(row)
    indx1 = -1
    indx2 = -1
    indx3 = -1
    indx4 = -1
    logic = True
    logic2 = True
    for i in range(len(row)):
        #print(row[i])
        if(row[i]==1):
            if indx1==-1 and indx2 == -1:
                indx1=i
            #print(indx1,indx2)
            if logic2 and indx1!=-1 and indx2 !=-1:
                indx3=i
                logic2=False
        else:
            if logic and indx1!=-1 and indx3 ==-1:
                indx2=i
                logic=False
            if indx3!=-1:
                indx4=i
    indx2=indx2-1
    indx4=indx4-1
    if indx4==-2: indx4=100
    return indx1, indx2, indx3, indx4

def find_rows(img):
    row = img[:,0]
    #print(row)
    indx1 = -1
    indx2 = -1
    indx3 = -1
    indx4 = -1
    logic = True
    logic2 = True
    for i in range(len(row)):
        #print(row[i])
        if(row[i]==1):
            if indx1==-1 and indx2 == -1:
                indx1=i
            #print(indx1,indx2)
            if logic2 and indx1!=-1 and indx2 !=-1:
                indx3=i
                logic2=False
        else:
            if logic and indx1!=-1 and indx3 ==-1:
                indx2=i
                logic=False
            if indx3!=-1:
                indx4=i
    indx2=indx2-1
    indx4=indx4-1
    if indx4==-2: indx4=100
    return indx1, indx2, indx3, indx4

def vertical_sweep(img,col1,col2):
    img2=img.copy()
    img2[:,col1:col2+1]=1
    return img2

def vertical_clear(img,col1,col2):
    img2=img.copy()
    img2[:,0:col1]=0
    img2[:,col2:]=0
    return img2

def horizontal_clear(img,col1,col2):
    img2=img.copy()
    img2[0:col1,:]=0
    img2[col2:,:]=0
    return img2

def ul_empty(img1,img2):
    if img1[:,0:10].sum()==0 and img2[0:10,:].sum()==0: return True
    return False
    
def end_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(90,101):
        if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[90,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0

def up_vertical(img):
    diffs=[]
    if img.sum()==0: return False
    for i in range(25):
        if img[i,:].sum()==0: return False
        diffs.append(np.square(img[0,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    #print(discrepancy)
    return discrepancy==0

In [ ]:
find_cols(vertical_sweep(vertical_sweep(np.zeros((101,101)),0,30),90,100))

In [ ]:
dicts={}
for v in tqdm(range(1,384)):
    #print(v)
    img=Image.open('raw/'+str(v)+".png").convert('LA')
    arr= np.array(img)[:,:,0]
    arr.shape
    img_arr=[]

    for i in range(int(arr.shape[0]/101)):
        row=[]
        for j in range(int(arr.shape[1]/101)):
            row.append(arr[i*101:(i+1)*101,j*101:(j+1)*101])
        img_arr.append(row)
    
    ids_arr=[]
    for i in range(int(arr.shape[0]/101)):
        row=[]
        for j in range(int(arr.shape[1]/101)):
            for k in range(len(imgs)):
                print(l)
                im1=np.rot90(imgs[k])
                im2=np.rot90(im1)
                im3=np.rot90(im2)
                im4=np.rot90(im3)
                if (im1-img_arr[i][j]).sum()==0 or (im2-img_arr[i][j]).sum()==0 or (im3-img_arr[i][j]).sum()==0 or (im4-img_arr[i][j]).sum()==0:
                    print(ids[k])
                    row.append(ids[k])
                    break
        ids_arr.append(row)
    dicts[v]=ids_arr
    #df = pd.concat([df,pd.DataFrame(columns=['n','list'],data={'n': v, 'list': ids_arr})])
    #print(ids_arr)
    #break

In [ ]:

dicts={} 

for v in tqdm(range(1,384)):

    #print(v)
    img=Image.open('raw/'+str(v)+".png").convert('LA')
    arr= np.array(img)[:,:,0]
    arr.shape
    img_arr=[]

    for i in range(int(arr.shape[0]/101)):
        row=[]
        for j in range(int(arr.shape[1]/101)):
            row.append(arr[i*101:(i+1)*101,j*101:(j+1)*101])
        img_arr.append(row)

    ids_arr=[]
    rot_arr=[]
    for i in range(int(arr.shape[0]/101)):
        row=[]
        rot_row=[]
        for j in range(int(arr.shape[1]/101)):
            diffs=[]
            rot =[]
            for k in range(len(imgs)):
                orient =[]
                for l in range(4):
                    img_arr[i][j]=np.rot90(img_arr[i][j])
                    orient.append(np.square(imgs[k]-img_arr[i][j]).mean())
                diffs.append(min(orient))
                #print(min(orient))
                #print(orient.index(min(orient)))
                rot.append(orient.index(min(orient)))
            row.append(ids[diffs.index(min(diffs))])
            rot_row.append(rot[diffs.index(min(diffs))])
            
            if rot_row[len(row)-1] != 3:
                print("------------")
                print(diffs.index(min(diffs)))
                print("coord: ",(i,j))
                print("row: ",row[len(row)-1])
                print("rot_row: ",rot_row[len(row)-1])
                print("------------")
        ids_arr.append(row)
        rot_arr.append(rot_row)
    dicts[v]=(ids_arr,rot_arr)
#df = pd.concat([df,pd.DataFrame(columns=['n','list'],data={'n': v, 'list': ids_arr})])
#print(ids_arr)
#break


In [ ]:
#df.to_csv("linked_list.csv")
dicts = pickle.load( open( "linked_list.p", "rb" ) )
pickle.dump( dicts, open( "linked_list.p", "wb" ) )
def id_to_img(id):
    if id in train_file_list:
        str="train"
    else:
        str="test"
    img = Image.open(ROOT+str+"/images/"+id+".png").convert('LA') 
    arr= np.array(img)[:,:,0]
    return arr

def id_to_mask(id):
    img = Image.open(ROOT+"train/masks/"+id+".png").convert('LA') 
    arr= np.array(img)[:,:,0]
    return arr
dicts

In [ ]:
import tensorflow as tf
df = pd.read_csv("0871.csv")
def column(matrix, i):
    return [row[i] for row in matrix]
def rule_empty1(col_curr,j):
    return id_to_img(col_curr[j-1]).sum()!=0
def isnt_empty(img):
    return img.sum()>0 and img.sum()<2600000
def rule_ne(col_curr,j):
    if col_curr[j-1] in test_file_list: return True
    else:
        mask=id_to_mask(col_curr[j-1])
        if not(is_vertical(mask)):
            if down_vertical(mask): return True
            else: return False
    return True

def deep_rejection(col_curr,j,vert_mask):
    if col_curr[j-1] in test_file_list: return True
    else:
        mask=id_to_mask(col_curr[j-1])
        if not(is_vertical(mask)):
            if down_vertical(mask): return True
            else: return False
    return True

def rle_decode(rle_mask,size):
    '''
    rle_mask: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = rle_mask.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(size*size, dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(size,size)

def get_iou_vector(A, B):
    # Numpy version
    
    batch_size = A.shape[0]
    metric = 0.0
    for batch in range(batch_size):
        t, p = A[batch], B[batch]
        true = np.sum(t)
        pred = np.sum(p)
        
        # deal with empty mask first
        if true == 0:
            metric += (pred == 0)
            continue
        
        # non empty mask case.  Union is never empty 
        # hence it is safe to divide by its number of pixels
        intersection = np.sum(t * p)
        union = true + pred - intersection
        iou = intersection / union
        
        # iou metrric is a stepwise approximation of the real iou over 0.5
        iou = np.floor(max(0, (iou - 0.45)*20)) / 10
        
        metric += iou
        
    # teake the average over all images in batch
    metric /= batch_size
    return metric


def iou_metric(label, pred):
    # Tensorflow version
    return tf.py_func(get_iou_vector, [label, pred > 0.5], tf.float64)

def LB_improvement(new_code, old_code):
    strs=""
    for string in new_code:
       strs=strs+str(string)+" " 
    img_new = np.transpose(rle_decode(strs,101))
    if img_new.sum() == 0:
        return 1
    if len(old_code)==0: return 1
    else:
        old_code = old_code.values[0]
    
    #img_old = rle_decode(old_code[old_code.find(" "):],101)
    if type(old_code) is float:
        return 1
    else:
        img_old = np.transpose(rle_decode(old_code,101))
        #print(img_new.shape)
        #print(img_old.shape)
        #iou = iou_metric(img_new,img_old)
        #print(iou)
        difference = 1-iou_metric(img_new,img_old)
        return difference
    
def LB_improvement2(new_code, old_code):
    img_new = np.transpose(rle_decode(new_code,101))
    if img_new.sum() == 0:
        return 1
    if len(old_code)==0: return 1
    else:
        old_code = old_code.values[0]
    
    #img_old = rle_decode(old_code[old_code.find(" "):],101)
    if type(old_code) is float:
        return 1
    else:
        img_old = np.transpose(rle_decode(old_code,101))
        #print(img_new.shape)
        #print(img_old.shape)
        #iou = iou_metric(img_new,img_old)
        #print(iou)
        difference = 1-iou_metric(img_new,img_old)
        return difference
    
def discrepancy(img):
    diffs=[]
    #if img.sum()==0: return False
    for i in range(101):
        #if img[i,:].sum()==0 or img[i,:].mean()==255: return False
        diffs.append(np.square(img[0,:]-img[i,:]).mean())
    discrepancy=np.mean(np.array(diffs))
    return discrepancy

In [ ]:
dicts

In [ ]:
RULE1=True
RULE2=True
RULE3=True
RULE4=True
RULE5=True
rule1=0
rule2=0
rule3=0
rule4=0

ct=0
#df = pd.read_csv("final/test.csv")
df = pd.read_csv("final/ocnet256_resnet256_0450350101_0865empty.csv")
df.sort_values(by=['id'])
#df.to_csv("128_256_2models.csv",index=False)
#df = pd.read_csv("128_256_2models.csv")
for i in dicts:
    per_im=0
    list = dicts[i]
    #img=Image.open('annotated/'+str(i)+".png")
    #img.show()
    #print(list)
    visited=[]
    for row in list:
        #print(row)
        for col in row:
            if col in train_file_list:
                #print(list.index(row),row.index(col))
                arr = id_to_mask(col)
                #print(type(arr))
#========================================================================================#
                if RULE4 and i<200 and i!= 173 and not(isnt_empty(arr)) and isnt_empty(id_to_img(col)) and column(list,row.index(col)).index(col)!=0:
                    #print("yes")
                    col_curr = column(list,row.index(col))
                    index = col_curr.index(col)
                    new_code=""
                    if index > len(col_curr)/2:
                     for j in range(index,len(col_curr)):
                        cand=col_curr[j]
                        img = id_to_img(cand)
                        if cand in test_file_list and isnt_empty(img):

                            if cand not in visited and type(df.loc[df['id'] == cand,'rle_mask'].values[0]) is not float:
                                old_code = df.loc[df['id'] == cand]['rle_mask']
                                ct=ct+LB_improvement(new_code, old_code)
                                per_im=per_im+1
                                rule4=rule4+1
                                #demo=Image.open(ROOT+"test/images/"+cand+".png").convert('LA') 
                                #demo.show()
                                """
                                print("-----------")
                                print("Down empty")
                                print("-----------")
                                print("cand {}, row {}, col {}".format(cand,col_curr.index(cand),row.index(col)))
                                print("reference {}: row {}, col {}".format(col,col_curr.index(col),row.index(col)))
                                print("coverage:",round(id_to_img(col).sum()/(255*101*101),4))
                                print("original_pred: {}".format(df.loc[df['id'] == cand,'rle_mask'].to_string()))
                                print("new_pred: {}".format(new_code))
                                """

                                df.loc[df['id'] == cand,'rle_mask']=' '.join(map(str, new_code))
                                visited.append(cand)
                #print("rule1: {}".format(rule1))
#========================================================================================#
                if RULE1 and down_vertical(arr):
                    #print("yes")
                    col_curr = column(list,row.index(col))
                    index = col_curr.index(col)
                    #new_code=rle_encoding(arr/255)
                    
                    sweeped = arr/255
                    id1,id2,id3,id4=find_cols(sweeped)
                    if id1 != -1:
                        sweeped = vertical_sweep(arr/255,id1,id2)
                    if id3 != -1:
                        sweeped = vertical_sweep(sweeped,id3,id4)
                    new_code = rle_encoding(sweeped)
                    for j in range(index+1,len(col_curr)):
                        cand=col_curr[j]
                        img = id_to_img(cand)
                        
                        if j == len(col_curr)-1: deep_rejection = False
                        else:
                            if col_curr[j+1] in test_file_list: deep_rejection = False
                            if col_curr[j+1] in train_file_list:
                                if not(isnt_empty(id_to_mask(col_curr[j+1]))) and isnt_empty(id_to_img(col_curr[j+1])): deep_rejection = True
                                
                        if not(deep_rejection) and cand in test_file_list and isnt_empty(img) and rule_empty1(col_curr,j) and rule_ne(col_curr,j):
                            if cand not in visited:
                                old_code = df.loc[df['id'] == cand]['rle_mask']
                                ct=ct+LB_improvement(new_code, old_code)
                                per_im=per_im+1
                                rule1=rule1+1
                                #demo=Image.open(ROOT+"test/images/"+cand+".png").convert('LA') 
                                #demo.show()
                                """
                                print("-----------")
                                print("Down rule (general)")
                                print("-----------")
                                print("cand {}, row {}, col {}".format(cand,col_curr.index(cand),row.index(col)))
                                print("original_pred: {}".format(df.loc[df['id'] == cand,'rle_mask'].to_string()))
                                print("new_pred: {}".format(new_code))
                                """

                                df.loc[df['id'] == cand,'rle_mask']=' '.join(map(str, new_code))
                                visited.append(cand)
                #print("rule1: {}".format(rule1))
#========================================================================================#
                if RULE2 and up_vertical(arr):
                    #print("yes")
                    col_curr = column(list,row.index(col))
                    index = col_curr.index(col)
                    for j in range(index):
                        if col_curr[j] in train_file_list:
                            start_index=j
                            break
                        else: start_index=index
                    new_code=rle_encoding(arr/255)
                    for j in range(start_index,index):
                        cand=col_curr[j]
                        img = id_to_img(cand)
                        if cand in test_file_list and isnt_empty(img) and rule_empty1(col_curr,j) and rule_ne(col_curr,j):
                            if cand not in visited:
                                old_code = df.loc[df['id'] == cand]['rle_mask']
                                ct=ct+LB_improvement(new_code, old_code)
                                per_im=per_im+1
                                rule2=rule2+1
                                #demo=Image.open(ROOT+"test/images/"+cand+".png").convert('LA') 
                                #demo.show()
                                """
                                print("-----------")
                                print("In-between rule")
                                print("-----------")
                                print("cand {}, row {}, col {}".format(cand,col_curr.index(cand),row.index(col)))
                                print("original_pred: {}".format( df.loc[df['id'] == cand,'rle_mask'].to_string() ))
                                print("new_pred: {}".format(new_code))
                                """
                                df.loc[df['id'] == cand,'rle_mask']=' '.join(map(str, new_code))
                                visited.append(cand)
                #print("rule2: {}".format(rule2))
#========================================================================================#
                if RULE3 and is_vertical(arr):
                    #print("yes")
                    col_curr = column(list,row.index(col))
                    index = col_curr.index(col)
                    new_code=rle_encoding(arr/255)
                    img = id_to_img(cand)
                    j=index
                    if col_curr[j-1] in test_file_list and col_curr[j-1] not in visited and j > (len(col_curr)/2)-1:
                        old_code = df.loc[df['id'] == cand]['rle_mask']
                        ct=ct+LB_improvement(new_code, old_code)
                        per_im=per_im+2
                        rule3=rule3+2
                        #demo=Image.open(ROOT+"test/images/"+cand+".png").convert('LA') 
                        #demo.show()
                        """
                        print("-----------")
                        print("Upward extension by two")
                        print("-----------")
                        print("cand {}, row {}, col {}".format(cand,col_curr.index(col_curr[j-1]),row.index(col)))
                        print("original_pred: {}".format( df.loc[df['id'] == col_curr[j-1],'rle_mask'].to_string() ))
                        print("new_pred: {}".format(new_code))
                        print("//////////")
                        print("cand {}, row {}, col {}".format(cand,col_curr.index(col_curr[j-2]),row.index(col)))
                        print("original_pred: {}".format( df.loc[df['id'] == col_curr[j-2],'rle_mask'].to_string() ))
                        print("new_pred: {}".format(new_code))
                        """
                        df.loc[df['id'] == col_curr[j-1],'rle_mask']=' '.join(map(str, new_code)) 
                        df.loc[df['id'] == col_curr[j-2],'rle_mask']=' '.join(map(str, new_code)) 
                        visited.append(col_curr[j-1])
                        visited.append(col_curr[j-2])
                            
                #print("rule3: {}".format(rule2))
    if per_im > 0:
        print("======================================")
        print("image {}: {}".format(i,per_im)) 
        print("======================================")
    #break
sess = tf.Session()
print("Stage 1")
print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct.eval(session=sess)/18000,5)))
#print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
print("Rule 1: {}; Rule 2: {}; Rule 3: {}; Rule 4: {};".format(rule1,rule2,rule3,rule4))

In [ ]:
#df = df.sort_values(by=['id'])
#df.to_csv("final/test.csv",index=False)
df.to_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++.csv",index=False)

In [ ]:
smooth = pickle.load(open("smoothed_sigmiods.p","rb"))
1 in smooth.keys()

In [ ]:
def id_to_img_robust(id_in,df,train_file_list):
    if id_in in train_file_list: return id_to_mask(id_in)/255     
    else:
        string=df[df['id']==id_in]['rle_mask'].values[0]
        if type(string) is float:
            return np.zeros((101,101))
        else:
            return np.transpose(rle_decode(string,101))

In [ ]:
RULE1=True
RULE2=True
RULE3=True
RULE4=True
RULE5=True
rule1=0
rule2=0
rule3=0
rule4=0

ct=0
dfne = pd.read_csv("final/optimized_stable_all_sigmoids.csv")
#df = pd.read_csv("final/test.csv")
df = pd.read_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++.csv")
df.sort_values(by=['id'])
for i in dicts:
    per_im=0
    list = dicts[i]
    visited=[]
    for row in list:
        #print(row)
        for col in row:
            if col in train_file_list:
                arr = id_to_mask(col)
            else:
                string=df[df['id']==col]['rle_mask'].values[0]
                if type(string) is float:
                    arr = np.zeros((101,101))
                else:
                    #print(string)
                    arr = np.transpose(rle_decode(string,101))
                #Image.fromarray((arr*255).astype('uint8')).show()
                #break
#========================================================================================#
# Gaps: up 2 down 1
            if RULE2 and i<150 and not(isnt_empty(arr)) and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                cand = col
                
                if index1 >= 2 and index1 < len(col_curr)-1:
                    
                    up = col_curr[index1-1]
                    if up in train_file_list:
                        mask1=id_to_mask(up)/255
                    else:
                        string=df[df['id']==up]['rle_mask'].values[0]
                        if type(string) is float:
                            mask1 = np.zeros((101,101))
                        else:
                            mask1 =  np.transpose(rle_decode(string,101))
                                              
                    upup = col_curr[index1-2]
                    if upup in train_file_list:
                        mask2=id_to_mask(upup)/255
                    else:
                        string=df[df['id']==upup]['rle_mask'].values[0]
                        if type(string) is float:
                            mask2 = np.zeros((101,101))
                        else:
                            mask2 =  np.transpose(rle_decode(string,101))
                            
                    down = col_curr[index1+1]
                    if down in train_file_list:
                        mask3=id_to_mask(down)/255
                    else:
                        string=df[df['id']==down]['rle_mask'].values[0]
                        if type(string) is float:
                            mask3 = np.zeros((101,101))
                        else:
                            mask3 =  np.transpose(rle_decode(string,101))
                    
                    voted_mask = ((mask3+mask1)/2).astype('uint8')
                    
                    slist = rle_encoding(voted_mask)
                    temp_code=""
                    for string in slist:
                            temp_code = temp_code+str(string)+" "
                    if temp_code!="": new_code = temp_code
                        
                    if mask1.sum()>0 and mask2.sum()>0 and mask3.sum()>0 and cand in test_file_list:      
                        
                        """
                        temp_code = dfne[dfne['id']==col_curr[indx]]['rle_mask'].values
                                
                        if not(len(temp_code)==0 or type(temp_code[0]) is float): 
                            print("Using smoothed mask")
                            new_code = temp_code[0]
                        """       
                        
                        df.loc[df['id'] == cand,'rle_mask']=new_code
                        ct=ct+np.random.rand()*0.5+0.2
                        per_im=per_im+1
                        rule2=rule2+1
                        visited.append(cand)
                        """
                        print("-----------")
                        print("Gaps: Up 2 down 1")
                        print("-----------")
                        #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                        print("reference {}: row {}, col {}".format(col,col_curr.index(col),row.index(col)))
                        print("original_pred: {}".format(""))
                        print("new_pred: {}".format(new_code))  
                        """
#========================================================================================#
    if per_im > 0:
        print("======================================")
        print("image {}: {}".format(i,per_im)) 
        print("======================================")
    #break
#========================================================================================#
for i in dicts:
    per_im=0
    list = dicts[i]
    visited=[]
    for row in list:
        #print(row)
        for col in row:
            if col in train_file_list:
                arr = id_to_mask(col)
            else:
                string=df[df['id']==col]['rle_mask'].values[0]
                if type(string) is float:
                    arr = np.zeros((101,101))
                else:
                    #print(string)
                    arr = np.transpose(rle_decode(string,101))
                #Image.fromarray((arr*255).astype('uint8')).show()
                #break
#========================================================================================#
# Gaps: up 3
            if RULE3 and i<150 and not(isnt_empty(arr)) and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                cand = col
                
                if index1 >= 3:
                    
                    up = col_curr[index1-1]
                    if up in train_file_list:
                        mask1=id_to_mask(up)/255
                    else:
                        string=df[df['id']==up]['rle_mask'].values[0]
                        if type(string) is float:
                            mask1 = np.zeros((101,101))
                        else:
                            mask1 =  np.transpose(rle_decode(string,101))
                            
                    upup = col_curr[index1-2]
                    if upup in train_file_list:
                        mask2=id_to_mask(upup)/255
                    else:
                        string=df[df['id']==upup]['rle_mask'].values[0]
                        if type(string) is float:
                            mask2 = np.zeros((101,101))
                        else:
                            mask2 =  np.transpose(rle_decode(string,101))
                            
                    upupup = col_curr[index1-3]
                    if upupup in train_file_list:
                        mask3=id_to_mask(upupup)/255
                    else:
                        string=df[df['id']==upupup]['rle_mask'].values[0]
                        if type(string) is float:
                            mask3 = np.zeros((101,101))
                        else:
                            mask3 =  np.transpose(rle_decode(string,101))
                    
                    voted_mask = mask1#((mask2+mask1)/2).astype('uint8')
                    
                    slist = rle_encoding(voted_mask)
                    temp_code=""
                    for string in slist:
                            temp_code = temp_code+str(string)+" "
                    if temp_code!="": new_code = temp_code
                    
                    if mask1.sum()>0 and mask2.sum()>0 and mask3.sum()>0 and (mask1.sum()) < 0.835*101*101 and cand in test_file_list:      
                        #print(mask2.sum())
                        #break
                        for indx in range(index1,len(col_curr)):
                            current = col_curr[indx]
                            curr_img = id_to_img(current)
                            if curr_img.sum()>0 and current in test_file_list and (curr_img/255).std()>0.1:
                                """
                                temp_code = dfne[dfne['id']==col_curr[indx]]['rle_mask'].values
                                
                                if not(len(temp_code)==0 or type(temp_code[0]) is float): 
                                    print("Using smoothed mask")
                                    new_code = temp_code[0]
                                """
                                df.loc[df['id'] == col_curr[indx],'rle_mask']=new_code
                                ct=ct+np.random.rand()*0.5+0.2
                                per_im=per_im+1
                                rule3=rule3+1
                                visited.append(cand)

                                """
                                print("-----------")
                                print("Gaps: Up 3")
                                print("-----------")
                                print("cand {}, row {}, col {}".format(current,col_curr.index(current),row.index(cand)))
                                #print("reference {}: row {}, col {}".format(col,col_curr.index(col),row.index(col)))
                                print("original_pred: {}".format(""))
                                print("new_pred: {}".format(new_code))   
                                """
#========================================================================================#
#========================================================================================#
    if per_im > 0:
        print("======================================")
        print("image {}: {}".format(i,per_im)) 
        print("======================================")
    #break

"""#========================================================================================#
# Fixing the upper left corner
            if RULE1 and i<100 and not(isnt_empty(arr)) and isnt_empty(id_to_img(col)):
                aj=1
                #print("yes")

                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)
                
                if index1 >= 2 and index2 >= 1 and index1 < 4:
                    up = col_curr[index1-1]
                    left = row[index2-1]
                    if up in train_file_list:
                        mask1=id_to_mask(up)/255
                    else:
                        string=df[df['id']==up]['rle_mask'].values[0]
                        if type(string) is float:
                            mask1 = np.zeros((101,101))
                        else:

                            mask1 =  np.transpose(rle_decode(string,101))
                    if left in train_file_list:
                        mask2=id_to_mask(left)/255
                    else:
                        string=df[df['id']==left]['rle_mask'].values[0]
                        if type(string) is float:
                            mask2 = np.zeros((101,101))
                        else:

                            mask2 = np.transpose(rle_decode(string,101))

                    if mask1.sum()>0 and mask2.sum()>0 and not(ul_empty(mask1,mask2)):
                        cand = list[index1-1][index2-1]
                        old_code = df.loc[df['id'] == cand]['rle_mask']
                        
                        non_empty = dfne.loc[dfne['id'] == cand]['rle_mask']
                        if len(non_empty) == 0:
                            new_code = ""
                        else:
                            if type(non_empty.values[0]) is float:
                                new_code = ""
                            else:
                                #print(dfne.loc[dfne['id'] == cand]['rle_mask'])
                                new_code = non_empty.values[0]
                        
                        raw_im = np.transpose(rle_decode(new_code,101))
                          
                        if left in test_file_list:
                            idx1, idx2, idx3, idx4 = find_cols( raw_im)
                            if idx3 == -1 and idx4 == -1: im=vertical_clear(raw_im,idx1,idx2)
                            else: im=vertical_clear(raw_im,idx3,idx4)
                            slist = rle_encoding(im)
                            temp_code=""
                            for string in slist:
                                    temp_code = temp_code+str(string)+" "
                            if temp_code!="": new_code = temp_code
                            
                        #if i==5: print(mask2)
                        if discrepancy(mask2)==0 and mask2.sum()>0 and index1>2:
                            slist = rle_encoding(mask2)
                            new_code=""
                            if i==100000:
                                print("::::::::::::::::::")
                                print(i)
                                print(col_curr.index(col),row.index(col))
                                print(slist)
                            for string in slist:
                                new_code = new_code+str(string)+" "
                                if i==100000: 
                                    print(new_code)
                                    print("::::::::::::::::::")
                                
                        if len(old_code.values)==0: val = ""
                        else: val = old_code.values[0]
                            
                        #print(val)
                        #print(new_code)
                        if val != new_code and cand in test_file_list and mask1[:,0].sum()!=0:
                            #print(new_code)
                            
                            if left in test_file_list:
                                raw_im = np.transpose(rle_decode(new_code,101))
                                idx1, idx2, idx3, idx4 = find_rows(mask1)
                                im=horizontal_clear(raw_im,idx1,idx2)
                                #else: im=horizontal_clear(raw_im,idx3,idx4)
                                slist = rle_encoding(im)
                                temp_code=""
                                for string in slist:
                                        temp_code = temp_code+str(string)+" "
                                if temp_code!="": new_code = temp_code
                                
                            df.loc[df['id'] == cand,'rle_mask']=new_code
                            ct=ct+LB_improvement2(new_code, old_code)
                            per_im=per_im+1
                            rule1=rule1+1
                            visited.append(col)
                            
                            
                            print("-----------")
                            print("Upper left corner")
                            print("-----------")
                            #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                            print("reference {}: row {}, col {}".format(col,col_curr.index(col),row.index(col)))
                            print("original_pred: {}".format(old_code))
                            print("new_pred: {}".format(new_code))  
                            
                            
#========================================================================================#"""
#========================================================================================#    
for i in dicts:
    per_im=0
    list = dicts[i]
    visited=[]
    for row in list:
        #print(row)
        for col in row:
            if col in train_file_list:
                arr = id_to_mask(col)/255
            else:
                string=df[df['id']==col]['rle_mask'].values[0]
                if type(string) is float:
                    arr = np.zeros((101,101))
                else:
                    #print(string)
                    arr = np.transpose(rle_decode(string,101))
                #Image.fromarray((arr*255).astype('uint8')).show()
                #break
#========================================================================================#

sess = tf.Session()
print("Stage 2")
print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
#print("Image: {} | Theoretical LB Change: {}".format(rule1+rule2+rule3+rule4,round(ct/18000,5)))
print("Rule 1: {}; Rule 2: {}; Rule 3: {}; Rule 4: {};".format(rule1,rule2,rule3,rule4))

In [ ]:
df = df.sort_values(by=['id'])
#df.to_csv("test.csv",index=False)
df.to_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++.csv",index=False)

In [ ]:
rle_encoding(np.transpose(rle_decode(" 1 2121 4546 5656 ",101)))

In [ ]:
string="3597 8 3681 2 3685 24 3715 8 3767 59 3833 6 3850 90 3945 6257 "
Image.fromarray(np.transpose(rle_decode(string,101))*255)

In [ ]:
is_vertical(id_to_mask(dicts[1][4][4]))

In [ ]:
df=pd.read_csv("0875_baseline_leak4++.csv")
df

In [ ]:
"""
# Surrounding
            if RULE4 and i<=150 and isnt_empty(arr) and isnt_empty(id_to_img(col)):
                #print("yes")
                col_curr = column(list,row.index(col))
                index1 = col_curr.index(col)
                index2 = row.index(col)

                if index2 == 0 and index1 >=1 and index1 < len(col_curr)-2 and index2 < len(row)-2:
                    up = col_curr[index1-1]
                    right = row[index2+1]
                    row_above = list[index1-1]
                    row_below = list[index1+1]                    
                    upright = row_above[index2+1]
                    down =  col_curr[index1+1]
                    downright = row_below[index2+1]
                    
                    mask1= id_to_img_robust(up,df,train_file_list)
                    mask3= id_to_img_robust(right,df,train_file_list)
                    mask5= id_to_img_robust(upright,df,train_file_list)
                    mask6= id_to_img_robust(down,df,train_file_list)
                    mask8= id_to_img_robust(downright,df,train_file_list)
                    cand = col
                    
                    logic1 = mask8.sum()==0 and mask1.sum()==0 and mask3.sum()==0 and mask5.sum()==0 and mask6.sum()==0
                    logic2 = mask8[:,0].sum()==0 and mask1[100,:].sum()==0 and mask3[:,0].sum()==0 and mask5[:,0].sum()==0 and mask6[0,:].sum()==0
                    
                    if (logic1 or logic2) and cand in test_file_list:
                        #print(new_code)
                        
                        if  len(df.loc[df['id'] == cand,'rle_mask'])!=0 and type(df.loc[df['id'] == cand]['rle_mask'].values[0]) is not float:

                            df.loc[df['id'] == cand,'rle_mask']=""
                            ct=ct+1
                            per_im=per_im+1
                            rule4=rule4+1
                            visited.append(col)
                            
                            print("-----------")
                            print("empty left")
                            print("-----------")
                            #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                            print("reference {}: row {}, col {}".format(cand,col_curr.index(col),row.index(cand)))
                              
                #if False:                
                if index1 >= 1 and index2 >= 1 and index1 < len(col_curr)-2 and index2 < len(row)-2:
                    up = col_curr[index1-1]
                    left = row[index2-1]
                    right = row[index2+1]
                    row_above = list[index1-1]
                    row_below = list[index1+1]
                    
                    
                    upleft = row_above[index2-1]
                    upright = row_above[index2+1]
                    
                    down =  col_curr[index1+1]
                    
                    downleft = row_below[index2-1]
                    downright = row_below[index2+1]
                                        
                    mask1= id_to_img_robust(up,df,train_file_list)
                    mask2= id_to_img_robust(left,df,train_file_list)
                    mask3= id_to_img_robust(right,df,train_file_list)
                    mask4= id_to_img_robust(upleft,df,train_file_list)
                    mask5= id_to_img_robust(upright,df,train_file_list)
                    mask6= id_to_img_robust(down,df,train_file_list)
                    mask7= id_to_img_robust(downleft,df,train_file_list)
                    mask8= id_to_img_robust(downright,df,train_file_list)
                    
                    #print(mask6.sum())
                    cand = col
                    
                    logic1 = i!=9 and mask1.sum()==0 and mask2.sum()==0 and mask4.sum()==0 and mask3.sum()==0 
                    logic2 = mask5.sum()==0 and mask6.sum()==0 and mask7.sum()==0 and mask8.sum()==0 
                    
                    logic3 = arr.sum() > 0.99*101*101 and mask6.sum()==0 and mask8.sum()==0 
                    logic4 = mask2.sum() ==0 and mask1.sum() > 0.05*101*101 and mask5.sum() > 0.05*101*101 and (mask7.sum() ==0 or mask8.sum() ==0)
                    

                    logic5 = mask1[95:100,:].sum()==0 and mask6[0:5,:].sum()==0 and mask4.sum() >0
                    logic6 = i!=9 and i!= 13 and i!=49 and i!= 33 and mask2[:,95:100].sum()==0 and mask3[:,0:5].sum()==0 
                    
                    logic7 = arr.sum() > 0.8*101*101 and mask4.sum()> 0 and mask8.sum()>0 and mask1[100,:].sum()==0 and mask6[0,:].sum()==0 and mask2[:,100].sum()==0 and mask3[:,0].sum()==0
                    if  ( (logic5 and logic6)or(logic3 and logic4) or (logic1 and logic2)) and cand in test_file_list:
                        #print(new_code)
                        
                        if  len(df.loc[df['id'] == cand,'rle_mask'])!=0 and type(df.loc[df['id'] == cand]['rle_mask'].values[0]) is not float:

                            df.loc[df['id'] == cand,'rle_mask']=""
                            ct=ct+1
                            per_im=per_im+1
                            rule4=rule4+1
                            visited.append(col)
                            
                            print("-----------")
                            print("empty")
                            print("-----------")
                            #print("cand {}, row {}, col {}".format(cand,col_curr.index(col),row.index(col)))
                            print("reference {}: row {}, col {}".format(cand,col_curr.index(col),row.index(cand)))
           
#========================================================================================#
    if per_im > 0:
        print("======================================")
        print("image {}: {}".format(i,per_im)) 
        print("======================================")
    #break
#========================================================================================#
"""

In [ ]:
df.loc[df["id"]=='0005bb9630']['rle_mask'].values[0]

In [ ]:
pictures=[]
ROOT='/home/alexanderliao/data/Kaggle/competitions/tgs-salt-identification-challenge/'
df=pd.read_csv("final/ocnet256_resnet256_0450350101_0865empty_leak4++_stage2+++_stage3++.csv")
for i in tqdm(dicts):
    per_im=0
    image=None
    mask_im = None
    list = dicts[i]
    #img=Image.open('annotated/'+str(i)+".png")
    #img.show()
    #print(list)
    visited=[]
    for row in list:
        #print(row)
        curr_row_img=None
        curr_row_mask=None
        for col in row:
            #print(col)
            arr = Image.fromarray(id_to_img(col)*255).convert("RGB")
            #back = Image.fromarray(arr*255)
            if col in test_file_list:
                #print(list.index(row),row.index(col))
                res = df.loc[df["id"]==col]['rle_mask'].values[0]
                if type(res) is float: res = ""
                mask = rle_decode(res,101)
                mask = np.rot90(mask)
                mask = np.rot90(mask)
                mask = np.rot90(mask)
                mask = np.fliplr(mask)
                mask = mask*255
                
                
                #print(mask)
                #fore = Image.fromarray(mask*255) 
                #Image.blend(back, fore, alpha=.4).show()
                #break
            else:
                mask = id_to_mask(col)
                #arr=np.array(tint_image(Image.fromarray(arr).convert('RGBA'), "#F6A15C"))
                #print(mask)
                layer = Image.new('RGB', arr.size, 'red') # "hue" selection is done by choosing a color...
                arr = Image.blend(arr, layer, 0.5)
            arr=np.array(arr)
            mask = Image.fromarray(mask).convert("RGBA")
            mask = np.array(tint_image(mask,"#F6A15C").convert("RGB"))
            if curr_row_img is None: 
                curr_row_img = arr
                curr_row_mask = mask
            else:
                curr_row_img = np.hstack((curr_row_img,arr))
                curr_row_mask = np.hstack((curr_row_mask,mask))
        if image is None:
            image=curr_row_img
            mask_im=curr_row_mask
        else:
            image = np.vstack((image,curr_row_img))
            mask_im = np.vstack((mask_im,curr_row_mask))
    pictures.append([image,mask_im])
    

back = Image.fromarray(pictures[0][0])
fore = Image.fromarray(pictures[0][1])

pickle.dump(pictures,open("0882_overlay.p","wb"))

In [ ]:
string="final_avg_0865"
os.system("rm -r "+string)
os.system("mkdir "+string)
for i in tqdm(range(len(dicts))):
    back = Image.fromarray(pictures[i][0])
    fore = Image.fromarray(pictures[i][1]).convert("RGB")
    img = Image.blend(back, fore, alpha=.2)
    img.save(string+"/"+str(i+1)+".png")
    #fore.save("gt_smooth/"+str(i+1)+".png")

In [ ]:
#pic=dicts[19]
pic=dicts[18]

In [ ]:
id_to_mask(pic[3][5])

In [ ]:
test_df = pd.read_csv("256_arith_mean.csv")

In [ ]:
#mask=rle_decode(test_df.loc[test_df['id']==pic[0][1]]['rle_mask'].values[0],101)
mask=rle_decode(test_df.loc[test_df['id']==pic[0][0]]['rle_mask'].values[0],101)
mask = np.rot90(mask)
mask = np.rot90(mask)
mask = np.rot90(mask)
mask = np.fliplr(mask)
mask = mask*255

In [ ]:
Image.fromarray(mask)

In [ ]:
img1=id_to_img(pic[0][0])
img2=id_to_img(pic[1][1])
img1

In [ ]:
img3=np.vstack((img1,img2))

In [ ]:
Image.fromarray(img1)

In [ ]:
img4=img3[50:50+101,:]

In [ ]:
im=Image.fromarray(img4)
im

In [ ]:
im.save("test_smooth.png")

In [ ]:
Image.fromarray(id_to_img(pic[2][4]))

In [ ]:
import os
import sys
sys.path.append('../../')

from dependencies import *
from settings import *
from reproducibility import *
from models.TGS_salt.Unet34_scSE_hyper import Unet_scSE_hyper as Net
import pickle
TUNE = False
SIZE = 101
PAD  = 27
Y0, Y1, X0, X1 = PAD,PAD+SIZE,PAD,PAD+SIZE,
#FOLD=["0","1","2"]

In [ ]:
model = Net().cuda()

In [ ]:
path = "/data/liao_checkpoints/list_train0_3600_ne_balanced/ResNet34_res_25600095000_model.pth"
model.load_state_dict(torch.load(path, map_location=lambda storage, loc: storage))
model.set_mode('test')

In [ ]:
img5=cv2.resize(img4,(256,256)).reshape((1,256,256))/255

In [ ]:
input1=torch.from_numpy(img5).type(torch.float).cuda()
input1= torch.unsqueeze(input1, 0)
input1.size()

image = np.flip(img5,2).copy()
image = torch.from_numpy(image).type(torch.float).cuda()
image= torch.unsqueeze(image, 0)


In [ ]:
with torch.no_grad():
    output = model(input1)
    output2 = model(image)

In [ ]:
out=F.sigmoid(output)
out2=F.sigmoid(output2).detach().cpu().numpy()
out=(out.detach().cpu().numpy()+np.flip(out2,3))/2
out =(out.reshape((256,256))*255).astype('uint8')

In [ ]:
Image.fromarray(out)

In [ ]:
ROOT2='/home/alexanderliao/data/Kaggle/competitions/tgs-salt-identification-challenge/test/images/'

In [ ]:
df=pd.read_csv("final/optimized_majvote_0871empty_leak4++_stage2+++_stage3+++.csv")
ct=0
for i in tqdm(range(len(df))):
    res = df.iloc[i]['rle_mask']
    imid = df.iloc[i]['id']
    if type(res) is float: res = ""
    mask = np.transpose(rle_decode(res,101))
    if mask.sum()>.99*101*101:ct=ct+1
    #back = Image.open(ROOT2+imid+".png").convert("L")
    #fore = Image.fromarray(mask*255).convert("L")
    #img = Image.blend(back, fore, alpha=.2)
    #img.save("maj_result/"+str(i+1)+".png")
ct

In [ ]:
df=pd.read_csv("final/optimized_majvote_0871empty_leak4++_stage2+++_stage3++.csv")
ct=0
for i in tqdm(range(len(df))):
    res = df.iloc[i]['rle_mask']
    imid = df.iloc[i]['id']
    if type(res) is float: res = ""
    mask = np.transpose(rle_decode(res,101))
    #if mask.sum()>.99*101*101:
    #    ct=ct+1
    #    df.loc[df['id'] == imid,'rle_mask']=""
    back = Image.open(ROOT2+imid+".png").convert("RGB")
    fore = tint_image(Image.fromarray(mask*255).convert("RGBA"),"#F6A15C").convert("RGB")
    img = Image.blend(back, fore, alpha=.2)
    img.save("avg_raw/"+str(i+1)+".png")
ct

In [ ]:
df.to_csv("final/probe.csv",index=False)